# Final Project - PGA Stats - Notebook #1
### Isaac Meadow, Jackson Skahill, Alex Eichmann

In [2]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [4]:
# Open PGA Tour
browser = webdriver.Chrome()
browser.get("https://www.pgatour.com/stats/stat.101.html")
time.sleep(5)

# Scraping
rows = browser.find_elements(By.TAG_NAME, "tr")
data = []

for row in rows[1:100]:
    cells = row.find_elements(By.TAG_NAME, "td")
    if len(cells) >= 6:
        try:
            player = cells[2].text
            avg_distance = cells[3].text
            rounds = cells[5].text
            data.append([player, rounds, avg_distance])
        except:
            continue

browser.quit()

In [5]:
# Create Data Frame
tour_stats = pd.DataFrame(data, columns=["Player", "Rounds", "Avg Distance"])
tour_stats["Player"] = tour_stats["Player"].str.strip()
tour_stats["Rounds"] = pd.to_numeric(tour_stats["Rounds"], errors="coerce")
tour_stats["Avg Distance"] = pd.to_numeric(tour_stats["Avg Distance"], errors="coerce")
display(tour_stats.head())

,Player,Rounds,Avg Distance
0,Aldrich Potgieter,58.0,323.9
1,Niklas Norgaard,48.0,319.5
2,Rory McIlroy,48.0,317.5
3,Kurt Kitayama,68.0,314.9
4,Michael Thorbjornsen,66.0,314.9


In [6]:
# Other CSV File
tour_data = pd.read_csv("pgaTourData.csv")


tour_data = tour_data[["Player Name", "Rounds", "Fairway Percentage", "Year",
                       "Avg Distance", "gir", "Average Putts", 
                       "Average Scrambling", "Average Score", "Points"]]
tour_data.rename(columns={"Player Name": "Player"}, inplace=True)
tour_data["Player"] = tour_data["Player"].str.strip()
tour_data["Rounds"] = pd.to_numeric(tour_data["Rounds"], errors="coerce")
tour_data["Avg Distance"] = pd.to_numeric(tour_data["Avg Distance"], errors="coerce")
tour_data.dropna(inplace=True)
display(tour_data)

,Player,Rounds,Fairway Percentage,Year,Avg Distance,gir,Average Putts,Average Scrambling,Average Score,Points
0,Henrik Stenson,60.0,75.19,2018,291.5,73.51,29.93,60.67,69.617,868
1,Ryan Armour,109.0,73.58,2018,283.5,68.22,29.31,60.13,70.758,"1,006"
2,Chez Reavie,93.0,72.24,2018,286.5,68.67,29.12,62.27,70.432,"1,020"
3,Ryan Moore,78.0,71.94,2018,289.2,68.80,29.17,64.16,70.015,795
4,Brian Stuard,103.0,71.44,2018,278.9,67.12,29.11,59.23,71.038,421
...,...,...,...,...,...,...,...,...,...,...
1673,Phil Mickelson,76.0,52.66,2010,299.1,65.13,28.79,61.84,69.966,"1,629"
1674,John Daly,63.0,52.21,2010,305.7,65.66,29.78,53.53,71.697,97
1675,Jimmy Walker,82.0,51.29,2010,292.9,65.88,29.14,58.46,70.953,554
1676,Daniel Chopra,74.0,51.27,2010,295.9,61.64,28.88,56.16,72.194,142


In [7]:
# Merge the two datasets
all_tour_data = pd.merge(tour_data, tour_stats, on="Player", how="left")


all_tour_data["Rounds"] = all_tour_data["Rounds_y"].fillna(all_tour_data["Rounds_x"])
all_tour_data["Avg Distance"] = all_tour_data["Avg Distance_y"].fillna(all_tour_data["Avg Distance_x"])

# Drop Extra Columns
all_tour_data.drop(columns=["Rounds_x", "Rounds_y", "Avg Distance_x", "Avg Distance_y"], inplace=True)


display(all_tour_data.head())

,Player,Fairway Percentage,Year,gir,Average Putts,Average Scrambling,Average Score,Points,Rounds,Avg Distance
0,Henrik Stenson,75.19,2018,73.51,29.93,60.67,69.617,868,60.0,291.5
1,Ryan Armour,73.58,2018,68.22,29.31,60.13,70.758,"1,006",109.0,283.5
2,Chez Reavie,72.24,2018,68.67,29.12,62.27,70.432,"1,020",93.0,286.5
3,Ryan Moore,71.94,2018,68.80,29.17,64.16,70.015,795,78.0,289.2
4,Brian Stuard,71.44,2018,67.12,29.11,59.23,71.038,421,103.0,278.9


In [8]:
all_tour_data.to_csv("final_merged_data.csv", index=False)